In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))



df = pd.read_csv(os.path.join(os.getcwd(), "../../data/total_4am.csv"), index_col=0)
excel_path = "../../results/results.xlsx"


In [2]:
import pandas as pd
from performance import get_performance

def various_ma(interval_list, df):

    original_df = df.copy()  # Make a copy of the original DataFrame

    for interval in interval_list:
        for interval2 in interval_list:
            if interval < interval2:
                # break

                df = original_df.copy()  # Create a fresh copy for each interval

                df[f"{interval}_ma"] = df['close'].rolling(window=interval).mean()
                df[f'{interval2}_ma'] = df['close'].rolling(window=interval2).mean()

                # Implement RSI strategy for long positions only
                df['signal'] = 0  # Default to no position
                for i in range(200, len(df)):
                    # Buy condition
                    # if df['close'].iloc[i] >= df[f'{interval}_ma'].iloc[i]:
                    if df.loc[i, f'{interval}_ma'] >= df.loc[i, f'{interval2}_ma'] and df.loc[i-1, f'{interval}_ma'] < df.loc[i-1, f'{interval2}_ma']:
                        df.loc[i, 'signal'] = 1
                    # Sell condition
                    elif df.loc[i, f'{interval}_ma'] < df.loc[i, f'{interval2}_ma'] and df.loc[i-1, f'{interval}_ma'] >= df.loc[i-1, f'{interval2}_ma']:
                        df.loc[i, 'signal'] = -1

                # Manage positions
                df['position'] = 0
                holding_position = False

                for i in range(1, len(df)):
                    if df.loc[i, 'signal'] == 1 and not holding_position:
                        df.loc[i, 'position'] = 1
                        holding_position = True
                    elif df.loc[i, 'signal'] == -1 and holding_position:
                        df.loc[i, 'position'] = 0
                        holding_position = False
                    else:
                        df.loc[i, 'position'] = df['position'].iloc[i-1]

                # Calculate the strategy returns (only when in a long position)
                df['strategy_returns'] = df['position'].shift(1) * df['close'].pct_change()

                df['strategy_returns2'] = df['strategy_returns']
                
                for i in range(1, len(df)):
                    buy_price = df.loc[i-1, "close"]
                    buy_price_copy = buy_price
                    sell_price = df.loc[i, "close"]
                    sell_price_copy = sell_price
                    if df.loc[i-1, 'position'] == 1 and df.loc[i-1, 'signal'] == 1:
                        # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'])/(df.loc[i-1, 'position'] * 1.002) -1
                        buy_price = df.loc[i-1, "close"] * 1.002
                    if df.loc[i, 'position'] == 0 and df.loc[i-1, 'position'] != 0:
                        # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'] * 0.998)/(df.loc[i-1, 'position']) -1
                        sell_price = df.loc[i, "close"] * 0.998
                    
                    if buy_price == buy_price_copy and sell_price == sell_price_copy:
                        continue


                    df.loc[i, "strategy_returns2"] = sell_price/buy_price - 1

                df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

                df["cumulative_returns2"] = (1 + df['strategy_returns2']).cumprod()

                # Calculate the benchmark cumulative returns (buy and hold strategy)
                df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

                # Save the results to a CSV file
                df.to_csv(f"../../results/ma/ma_crossover({interval})_({interval2}).csv")

                # Get performance metrics and plot the results
                excel_path = "../../results/results.xlsx"
                get_performance(df=df, title=f"ma_crossover({interval})_({interval2})", add_to_excel=True, file_path=excel_path)

                
            df = original_df.copy()  # Create a fresh copy for each interval


In [3]:
interval_list = [5, 10, 20, 50, 90, 120, 200]
various_ma(interval_list=interval_list, df=df)

======Investment Summary======
Strategy        : ma_crossover(5)_(10)
total_return    : 53.55
cagr            : 16.62
mdd             : 51.57
total_return_w_fee : 23.23
cagr_w_fee      : 7.78
mdd_w_fee       : 56.18
investing_days  : 1018
======Investment Summary======
Strategy        : ma_crossover(5)_(20)
total_return    : 44.14
cagr            : 14.05
mdd             : 50.59
total_return_w_fee : 28.35
cagr_w_fee      : 9.39
mdd_w_fee       : 52.91
investing_days  : 1015
======Investment Summary======
Strategy        : ma_crossover(5)_(50)
total_return    : 53.27
cagr            : 18.42
mdd             : 41.23
total_return_w_fee : 44.34
cagr_w_fee      : 15.64
mdd_w_fee       : 42.74
investing_days  : 922
======Investment Summary======
Strategy        : ma_crossover(5)_(90)
total_return    : 48.09
cagr            : 17.69
mdd             : 42.18
total_return_w_fee : 41.15
cagr_w_fee      : 15.37
mdd_w_fee       : 42.98
investing_days  : 880
======Investment Summary======
Strategy     